In [1]:
import pandas as pd
import numpy as np

## Load data

In [2]:
df_transactions = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx', sheet_name='Transactions',header=1,
                                converters={
        'product_first_sold_date': lambda x: pd.to_datetime(x, unit='D', origin='1899-12-30')
    })
df_newcus = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx', sheet_name='NewCustomerList',header=1)
df_demo = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx', sheet_name='CustomerDemographic',header=1)
df_address = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx', sheet_name='CustomerAddress',header=1)

The product_first_sold_date column have contains excel serial date format which excel serial dates represent the number of days since the base date of January 1, 1900 (with adjustments for leap years). Stackoverflow suggested that the dat should begin at 1899-12-30, https://stackoverflow.com/questions/9574793/how-to-convert-a-python-datetime-datetime-to-excel-serial-date-number/9574948#9574948.

## Check loaded data

In [3]:
df_name = ['Transactions','NewCustomerList','CustomerDemographic','CustomerAddress']
df_all = [df_transactions,df_newcus,df_demo,df_address]

for name, df in zip(df_name, df_all):
    print(name)
    print(df.info())
    print()

Transactions
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   transaction_id           20000 non-null  int64         
 1   product_id               20000 non-null  int64         
 2   customer_id              20000 non-null  int64         
 3   transaction_date         20000 non-null  datetime64[ns]
 4   online_order             19640 non-null  float64       
 5   order_status             20000 non-null  object        
 6   brand                    19803 non-null  object        
 7   product_line             19803 non-null  object        
 8   product_class            19803 non-null  object        
 9   product_size             19803 non-null  object        
 10  list_price               20000 non-null  float64       
 11  standard_cost            19803 non-null  float64       
 12  product_first_sold_

- There are some missing values found in Transaction, NewCustomerList and CustomerDemographic
- Unnamed columns was found, I assumed that it have been used to calculate Rank column
- There are 1 missing record in CustomerAddress, assuming that CustomerDemographic and customerAddress was linked by customer ID.

## Check duplicate

In [4]:
df_name = ['Transactions','NewCustomerList','CustomerDemographic','CustomerAddress']
df_all = [df_transactions,df_newcus,df_demo,df_address]

for name, df in zip(df_name, df_all):
    print(name)
    print(df.duplicated().any())
    print()

Transactions
False

NewCustomerList
False

CustomerDemographic
False

CustomerAddress
False



There is no duplicated rows.

## Check unique values

In [5]:
df_name = ['Transactions','NewCustomerList','CustomerDemographic','CustomerAddress']
df_all = [df_transactions,df_newcus,df_demo,df_address]

for name, df in zip(df_name, df_all):
    print(name)
    print(df.nunique())
    print()

Transactions
transaction_id             20000
product_id                   101
customer_id                 3494
transaction_date             364
online_order                   2
order_status                   2
brand                          6
product_line                   4
product_class                  3
product_size                   3
list_price                   296
standard_cost                103
product_first_sold_date      100
dtype: int64

NewCustomerList
first_name                              940
last_name                               961
gender                                    3
past_3_years_bike_related_purchases     100
DOB                                     961
job_title                               184
job_industry_category                     9
wealth_segment                            3
deceased_indicator                        1
owns_car                                  2
tenure                                   23
address                                1000


## Check null value

In [6]:
for name, df in zip(df_name, df_all):
    print(name)
    print(df.isnull().sum())
    print()

Transactions
transaction_id               0
product_id                   0
customer_id                  0
transaction_date             0
online_order               360
order_status                 0
brand                      197
product_line               197
product_class              197
product_size               197
list_price                   0
standard_cost              197
product_first_sold_date    197
dtype: int64

NewCustomerList
first_name                               0
last_name                               29
gender                                   0
past_3_years_bike_related_purchases      0
DOB                                     17
job_title                              106
job_industry_category                  165
wealth_segment                           0
deceased_indicator                       0
owns_car                                 0
tenure                                   0
address                                  0
postcode                              

In [7]:
for name, df in zip(df_name, df_all):
    print(f"Summary for DataFrame: {name}")
    print("-" * 40)
    
    # Display null values for columns with missing data
    null_columns = df.columns[df.isnull().any()]
    print("Null Values:")
    print(df[null_columns].isnull().sum())
    print("\n" + "-" * 40 + "\n")

Summary for DataFrame: Transactions
----------------------------------------
Null Values:
online_order               360
brand                      197
product_line               197
product_class              197
product_size               197
standard_cost              197
product_first_sold_date    197
dtype: int64

----------------------------------------

Summary for DataFrame: NewCustomerList
----------------------------------------
Null Values:
last_name                 29
DOB                       17
job_title                106
job_industry_category    165
dtype: int64

----------------------------------------

Summary for DataFrame: CustomerDemographic
----------------------------------------
Null Values:
last_name                125
DOB                       87
job_title                506
job_industry_category    656
default                  302
tenure                    87
dtype: int64

----------------------------------------

Summary for DataFrame: CustomerAddress
------

## Overall missing values

In [8]:
for name, df in zip(df_name, df_all):
    print(f"Null summary of {name}")
    print("-" * 40)
    
    # Display null values for columns with missing data
    null_summary = df.isnull().sum()
    total_values = df.shape[0]
    percentage_null = (null_summary / total_values) * 100
    print("Percentage of Null Values:")
    print(percentage_null)
    print("\n" + "-" * 40 + "\n")

Null summary of Transactions
----------------------------------------
Percentage of Null Values:
transaction_id             0.000
product_id                 0.000
customer_id                0.000
transaction_date           0.000
online_order               1.800
order_status               0.000
brand                      0.985
product_line               0.985
product_class              0.985
product_size               0.985
list_price                 0.000
standard_cost              0.985
product_first_sold_date    0.985
dtype: float64

----------------------------------------

Null summary of NewCustomerList
----------------------------------------
Percentage of Null Values:
first_name                              0.0
last_name                               2.9
gender                                  0.0
past_3_years_bike_related_purchases     0.0
DOB                                     1.7
job_title                              10.6
job_industry_category                  16.5
wealth_s

## Check customer ID in CustomerDemographic and customerAddress

In [9]:
missing_records = df_address[~df_address['customer_id'].isin(df_demo['customer_id'])]
print("Missing Records in customer demographic:")
cus_id_address_diff = len(missing_records)
display(missing_records)

Missing Records in customer demographic:


,customer_id,address,postcode,state,country,property_valuation
3996,4001,87 Crescent Oaks Alley,2756,NSW,Australia,10
3997,4002,8194 Lien Street,4032,QLD,Australia,7
3998,4003,320 Acker Drive,2251,NSW,Australia,7


In [10]:
inconsistent_trans_cusID = df_transactions[~df_transactions['customer_id'].isin(df_demo['customer_id'])]
cus_id_trans_diff = len(inconsistent_trans_cusID)
display(inconsistent_trans_cusID)

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
8707,8708,0,5034,2017-10-07,0.0,Approved,Solex,Road,medium,medium,416.98,312.735016,2015-08-02
16700,16701,0,5034,2017-01-27,0.0,Approved,Norco Bicycles,Standard,medium,medium,360.40,270.299988,1997-08-25
17468,17469,0,5034,2017-01-03,0.0,Approved,OHM Cycles,Road,medium,medium,742.54,667.400024,1992-10-02


## Check customer ID in customerAddress that was skipped

In [11]:
cus_demo_id = df_demo['customer_id'].to_list()

# Generate a reference list of all numbers in the expected range
expected_numbers = list(range(1, max(cus_demo_id) + 1))

# Find the missing numbers
missing_numbers = list(set(expected_numbers) - set(cus_demo_id))
missing_cus_id_demo = len(missing_numbers)
# Display the missing numbers
print("Missing Numbers:", missing_numbers)

Missing Numbers: []


In [12]:
cus_add_id = df_address['customer_id'].to_list()

# Generate a reference list of all numbers in the expected range
expected_numbers = list(range(1, max(cus_add_id) + 1))

# Find the missing numbers
missing_numbers = list(set(expected_numbers) - set(cus_add_id))
missing_cus_id_address = len(missing_numbers)
# Display the missing numbers
print("Missing Numbers:", missing_numbers)

Missing Numbers: [10, 3, 22, 23]


## Data preprocessing

### Transaction dataset

In [13]:
df_transactions.describe()

,transaction_id,product_id,customer_id,transaction_date,online_order,list_price,standard_cost,product_first_sold_date
count,20000.000000,20000.00000,20000.000000,20000,19640.000000,20000.000000,19803.000000,19803
mean,10000.500000,45.36465,1738.246050,2017-07-01 14:08:05.280000,0.500458,1107.829449,556.046951,2004-07-31 18:38:13.834267392
min,1.000000,0.00000,1.000000,2017-01-01 00:00:00,0.000000,12.010000,7.210000,1991-01-21 00:00:00
25%,5000.750000,18.00000,857.750000,2017-04-01 00:00:00,0.000000,575.270000,215.140000,1997-08-25 00:00:00
50%,10000.500000,44.00000,1736.000000,2017-07-03 00:00:00,1.000000,1163.890000,507.580000,2004-08-17 00:00:00
75%,15000.250000,72.00000,2613.000000,2017-10-02 00:00:00,1.000000,1635.300000,795.100000,2011-05-09 00:00:00
max,20000.000000,100.00000,5034.000000,2017-12-30 00:00:00,1.000000,2091.470000,1759.850000,2016-12-06 00:00:00
std,5773.647028,30.75359,1011.951046,NaN,0.500013,582.825242,405.955660,NaN


In [14]:
# row with null value
rows_with_missing_values_trans = df_transactions[df_transactions.isnull().any(axis=1)]
print(len(rows_with_missing_values_trans))

555


In [15]:
df_transactions.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,2012-12-02
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,2014-03-03
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,1999-07-20
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,1998-12-16
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,2015-08-10


In [16]:
# Analyze each customer's historical data and fill null values based on their most common behavior (mode).
missing_online_order = df_transactions[df_transactions['online_order'].isnull()]
df_transactions['online_order'] = df_transactions.groupby('customer_id')['online_order'].transform(lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else False))
df_transactions.isnull().sum()

transaction_id               0
product_id                   0
customer_id                  0
transaction_date             0
online_order                 0
order_status                 0
brand                      197
product_line               197
product_class              197
product_size               197
list_price                   0
standard_cost              197
product_first_sold_date    197
dtype: int64

In [17]:
# brand
null_brand = df_transactions[df_transactions['brand'].isnull()]
null_brand

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
136,137,0,431,2017-09-23,0.0,Approved,NaN,NaN,NaN,NaN,1942.61,NaN,NaT
159,160,0,3300,2017-08-27,0.0,Approved,NaN,NaN,NaN,NaN,1656.86,NaN,NaT
366,367,0,1614,2017-03-10,0.0,Approved,NaN,NaN,NaN,NaN,850.89,NaN,NaT
406,407,0,2559,2017-06-14,1.0,Approved,NaN,NaN,NaN,NaN,710.59,NaN,NaT
676,677,0,2609,2017-07-02,0.0,Approved,NaN,NaN,NaN,NaN,1972.01,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19340,19341,0,443,2017-12-26,1.0,Approved,NaN,NaN,NaN,NaN,744.54,NaN,NaT
19383,19384,0,2407,2017-06-11,0.0,Approved,NaN,NaN,NaN,NaN,1098.18,NaN,NaT
19793,19794,0,2860,2017-01-13,0.0,Approved,NaN,NaN,NaN,NaN,868.56,NaN,NaT
19859,19860,0,2468,2017-06-24,1.0,Approved,NaN,NaN,NaN,NaN,1497.43,NaN,NaT


In [18]:
proportion_null_brand = (len(null_brand) / len(df_transactions)) * 100
print(proportion_null_brand)

0.985


Only the list_price value is not strong enough to determine which brand, product_line, product_class, product_size, and standard_cost are. Therefore, I will drop the row where brand is null values. However, it can be investigated to find out the actual values of them in the future.

In [19]:
df_transactions.dropna(subset=['brand'], inplace=True)
df_transactions

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,2012-12-02
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,2014-03-03
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,1999-07-20
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,1998-12-16
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,2015-08-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,51,1018,2017-06-24,1.0,Approved,OHM Cycles,Standard,high,medium,2005.66,1203.40,2003-07-21
19996,19997,41,127,2017-11-09,1.0,Approved,Solex,Road,medium,medium,416.98,312.74,1997-05-10
19997,19998,87,2284,2017-04-14,1.0,Approved,OHM Cycles,Standard,medium,medium,1636.90,44.71,2010-08-20
19998,19999,6,2764,2017-07-03,0.0,Approved,OHM Cycles,Standard,high,medium,227.88,136.73,2004-08-17


In [20]:
df_transactions.isnull().sum()

transaction_id             0
product_id                 0
customer_id                0
transaction_date           0
online_order               0
order_status               0
brand                      0
product_line               0
product_class              0
product_size               0
list_price                 0
standard_cost              0
product_first_sold_date    0
dtype: int64

## Calculate error in the transactions dataset

In [21]:
unique_error_rows = len(set(missing_online_order.index) | set(null_brand.index))
trans_null_row = len(rows_with_missing_values_trans)
# transaction_error = (unique_error_rows / len(df_transactions)) * 100
transaction_error = ((len(df_transactions) - trans_null_row) / len(df_transactions)) * 100
print(f'The percentage of rows without missing values is: {round(transaction_error, 2)}')

The percentage of rows without missing values is: 97.2


### NewCustomerList dataset

In [22]:
df_newcus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 23 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   first_name                           1000 non-null   object 
 1   last_name                            971 non-null    object 
 2   gender                               1000 non-null   object 
 3   past_3_years_bike_related_purchases  1000 non-null   int64  
 4   DOB                                  983 non-null    object 
 5   job_title                            894 non-null    object 
 6   job_industry_category                835 non-null    object 
 7   wealth_segment                       1000 non-null   object 
 8   deceased_indicator                   1000 non-null   object 
 9   owns_car                             1000 non-null   object 
 10  tenure                               1000 non-null   int64  
 11  address                        

In [23]:
# convert DOB to datetime
df_newcus['DOB'] = pd.to_datetime(df_newcus['DOB'], errors='coerce')

In [24]:
# Count the number of rows with at least one missing value
rows_with_missing_values = df_newcus[df_newcus.isnull().any(axis=1)]
count_missing_rows = len(rows_with_missing_values)
print(count_missing_rows)

285


In [25]:
# filter the categorical columns and replace the null value with 'unknow'
text_col_newcus = df_newcus.select_dtypes(include=['object']).columns
text_cols_with_null = df_newcus[text_col_newcus].columns[df_newcus[text_col_newcus].isnull().any()]
print(text_cols_with_null)

Index(['last_name', 'job_title', 'job_industry_category'], dtype='object')


In [26]:
for col in text_cols_with_null:
    df_newcus[col].fillna('unknown', inplace=True)

In [27]:
df_newcus[df_newcus['DOB'].isnull()]

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,...,state,country,property_valuation,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Rank,Value
59,Normy,Goodinge,U,5,NaT,Associate Professor,IT,Mass Customer,N,No,...,VIC,Australia,5,0.78,0.7800,0.78000,0.663000,57,57,1.375000
226,Hatti,Carletti,U,35,NaT,Legal Assistant,IT,Affluent Customer,N,Yes,...,NSW,Australia,9,0.92,1.1500,1.43750,1.437500,226,226,1.112500
324,Rozamond,Turtle,U,69,NaT,Legal Assistant,IT,Mass Customer,N,Yes,...,VIC,Australia,3,1.08,1.3500,1.35000,1.147500,324,324,1.010000
358,Tamas,Swatman,U,65,NaT,Assistant Media Planner,Entertainment,Affluent Customer,N,No,...,QLD,Australia,8,1.01,1.0100,1.01000,1.010000,358,358,0.980000
360,Tracy,Andrejevic,U,71,NaT,Programmer II,IT,Mass Customer,N,Yes,...,VIC,Australia,7,0.95,1.1875,1.18750,1.009375,361,361,0.977500
374,Agneta,McAmish,U,66,NaT,Structural Analysis Engineer,IT,Mass Customer,N,No,...,QLD,Australia,6,0.67,0.6700,0.67000,0.569500,375,375,0.960000
434,Gregg,Aimeric,U,52,NaT,Internal Auditor,IT,Mass Customer,N,No,...,VIC,Australia,5,0.83,0.8300,0.83000,0.705500,433,433,0.906250
439,Johna,Bunker,U,93,NaT,Tax Accountant,IT,Mass Customer,N,Yes,...,VIC,Australia,6,0.90,1.1250,1.40625,1.195312,436,436,0.903125
574,Harlene,Nono,U,69,NaT,Human Resources Manager,IT,Mass Customer,N,No,...,NSW,Australia,7,0.76,0.7600,0.76000,0.646000,575,575,0.796875
598,Gerianne,Kaysor,U,15,NaT,Project Manager,IT,Affluent Customer,N,No,...,NSW,Australia,11,1.08,1.0800,1.35000,1.350000,599,599,0.775000


In [28]:
# fill with mean value
mean_dob = df_newcus['DOB'].mean()
df_newcus['DOB'].fillna(mean_dob, inplace=True)

In [29]:
# calculate age
current_date = pd.to_datetime('today')
df_newcus['age'] = (current_date - df_newcus['DOB']).dt.days // 365.25
df_newcus['age'] = df_newcus['age'].astype(int)

In [30]:
df_newcus

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,...,country,property_valuation,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Rank,Value,age
0,Chickie,Brister,Male,86,1957-07-12,General Manager,Manufacturing,Mass Customer,N,Yes,...,Australia,6,0.56,0.7000,0.875000,0.743750,1,1,1.718750,66
1,Morly,Genery,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,N,No,...,Australia,11,0.89,0.8900,1.112500,0.945625,1,1,1.718750,53
2,Ardelis,Forrester,Female,10,1974-08-28,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,...,Australia,5,1.01,1.0100,1.010000,1.010000,1,1,1.718750,49
3,Lucine,Stutt,Female,64,1979-01-28,Account Representative III,Manufacturing,Affluent Customer,N,Yes,...,Australia,1,0.87,1.0875,1.087500,1.087500,4,4,1.703125,44
4,Melinda,Hadlee,Female,34,1965-09-21,Financial Analyst,Financial Services,Affluent Customer,N,No,...,Australia,9,0.52,0.5200,0.650000,0.650000,4,4,1.703125,58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Ferdinand,Romanetti,Male,60,1959-10-07,Paralegal,Financial Services,Affluent Customer,N,No,...,Australia,7,0.79,0.7900,0.790000,0.790000,996,996,0.374000,64
996,Burk,Wortley,Male,22,2001-10-17,Senior Sales Associate,Health,Mass Customer,N,No,...,Australia,10,0.76,0.7600,0.950000,0.807500,997,997,0.357000,22
997,Melloney,Temby,Female,17,1954-10-05,Budget/Accounting Analyst IV,Financial Services,Affluent Customer,N,Yes,...,Australia,2,0.85,1.0625,1.062500,1.062500,997,997,0.357000,69
998,Dickie,Cubbini,Male,30,1952-12-17,Financial Advisor,Financial Services,Mass Customer,N,Yes,...,Australia,2,1.09,1.3625,1.362500,1.158125,997,997,0.357000,70


In [31]:
df_newcus.isnull().sum()

first_name                             0
last_name                              0
gender                                 0
past_3_years_bike_related_purchases    0
DOB                                    0
job_title                              0
job_industry_category                  0
wealth_segment                         0
deceased_indicator                     0
owns_car                               0
tenure                                 0
address                                0
postcode                               0
state                                  0
country                                0
property_valuation                     0
Unnamed: 16                            0
Unnamed: 17                            0
Unnamed: 18                            0
Unnamed: 19                            0
Unnamed: 20                            0
Rank                                   0
Value                                  0
age                                    0
dtype: int64

In [32]:
df_newcus[226:227]

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,...,country,property_valuation,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Rank,Value,age
226,Hatti,Carletti,U,35,1971-04-20 14:05:14.954221768,Legal Assistant,IT,Affluent Customer,N,Yes,...,Australia,9,0.92,1.15,1.4375,1.4375,226,226,1.1125,52


In [33]:
df_newcus['age'].nlargest(5)

18     85
191    85
265    85
414    85
585    85
Name: age, dtype: int64

## Calculate error in the NewCustomerList dataset

In [34]:
NewCustomerList_error = ((len(df_newcus) - count_missing_rows)/len(df_newcus))*100 
print(round(NewCustomerList_error, 2))

71.5


### CustomerDemographic dataset

In [35]:
df_demo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   customer_id                          4000 non-null   int64  
 1   first_name                           4000 non-null   object 
 2   last_name                            3875 non-null   object 
 3   gender                               4000 non-null   object 
 4   past_3_years_bike_related_purchases  4000 non-null   int64  
 5   DOB                                  3913 non-null   object 
 6   job_title                            3494 non-null   object 
 7   job_industry_category                3344 non-null   object 
 8   wealth_segment                       4000 non-null   object 
 9   deceased_indicator                   4000 non-null   object 
 10  default                              3698 non-null   object 
 11  owns_car                      

In [36]:
# convert DOB to datetime
df_demo['DOB'] = pd.to_datetime(df_demo['DOB'], errors='coerce')

## Calculate error in the customerDemographic dataset

In [37]:
# Count the number of rows with at least one missing value
rows_with_missing_values_demo = df_demo[df_demo.isnull().any(axis=1)]
count_missing_rows_demo = len(rows_with_missing_values_demo)
print(f"Count of missing rows in df_demo: {count_missing_rows_demo}")

percentage_missing_rows_demo = ((len(df_demo) - count_missing_rows_demo) / len(df_demo)) * 100
print(f"Percentage of rows with missing values in df_demo: {round(percentage_missing_rows_demo, 2)}")

Count of missing rows in df_demo: 1370
Percentage of rows with missing values in df_demo: 65.75


In [38]:
text_col_demo = df_demo.select_dtypes(include=['object']).columns
text_cols_with_null = df_demo[text_col_demo].columns[df_demo[text_col_demo].isnull().any()]
print(text_cols_with_null)

Index(['last_name', 'job_title', 'job_industry_category', 'default'], dtype='object')


In [39]:
for col in text_cols_with_null:
    df_demo[col].fillna('unknown', inplace=True)

In [40]:
# fill with mean value
mean_dob = df_demo['DOB'].mean()
df_demo['DOB'].fillna(mean_dob, inplace=True)

In [41]:
# calculate age
current_date = pd.to_datetime('today')
df_demo['age'] = (current_date - df_demo['DOB']).dt.days // 365.25
df_demo['age'] = df_demo['age'].astype(int)

In [42]:
# fill tenure with mean value
df_demo['tenure'].fillna(df_demo['tenure'].mean(), inplace=True)

In [43]:
df_demo.isnull().sum()

customer_id                            0
first_name                             0
last_name                              0
gender                                 0
past_3_years_bike_related_purchases    0
DOB                                    0
job_title                              0
job_industry_category                  0
wealth_segment                         0
deceased_indicator                     0
default                                0
owns_car                               0
tenure                                 0
age                                    0
dtype: int64

## Check values consistency

In [44]:
transactions_categorical = df_transactions.select_dtypes(include=['object']).columns
for col in transactions_categorical:
    print(df_transactions[col].value_counts())
    print()

online_order
0.0    9915
1.0    9888
Name: count, dtype: int64

order_status
Approved     19625
Cancelled      178
Name: count, dtype: int64

brand
Solex             4253
Giant Bicycles    3312
WeareA2B          3295
OHM Cycles        3043
Trek Bicycles     2990
Norco Bicycles    2910
Name: count, dtype: int64

product_line
Standard    14176
Road         3970
Touring      1234
Mountain      423
Name: count, dtype: int64

product_class
medium    13826
high       3013
low        2964
Name: count, dtype: int64

product_size
medium    12990
large      3976
small      2837
Name: count, dtype: int64



In [45]:
transactions_numerical = df_transactions.select_dtypes(exclude=['object']).columns
for col in transactions_numerical:
    print(df_transactions[col].value_counts())
    print()

transaction_id
1        1
13324    1
13331    1
13330    1
13329    1
        ..
6654     1
6653     1
6652     1
6651     1
20000    1
Name: count, Length: 19803, dtype: int64

product_id
0      1181
3       354
1       311
35      268
38      267
       ... 
71      137
8       136
16      136
100     130
47      121
Name: count, Length: 101, dtype: int64

customer_id
1068    14
2183    14
2476    14
2548    13
2072    13
        ..
1846     1
1757     1
1865     1
774      1
3464     1
Name: count, Length: 3494, dtype: int64

transaction_date
2017-08-18    82
2017-02-14    81
2017-10-15    74
2017-01-31    72
2017-12-19    70
              ..
2017-06-14    38
2017-12-07    37
2017-03-29    35
2017-09-25    35
2017-10-19    32
Name: count, Length: 364, dtype: int64

list_price
2091.47    465
1403.50    396
71.49      274
1231.15    235
1890.39    233
          ... 
1172.78    157
358.39     141
1227.34    126
202.62     124
1036.59    114
Name: count, Length: 100, dtype: int64

stand

In [46]:
df_transactions['transaction_id'].duplicated().any()

False

In [47]:
newcus_categorical = df_newcus.select_dtypes(include=['object']).columns
for col in newcus_categorical:
    print(df_newcus[col].value_counts())
    print()

first_name
Rozamond     3
Dorian       3
Mandie       3
Inglebert    2
Ricki        2
            ..
Diego        1
Lucilia      1
Eddy         1
Caron        1
Sylas        1
Name: count, Length: 940, dtype: int64

last_name
unknown      29
Eade          2
Sissel        2
Borsi         2
Shoesmith     2
             ..
O'Moylane     1
Axtens        1
Moxted        1
Conrad        1
Duffill       1
Name: count, Length: 962, dtype: int64

gender
Female    513
Male      470
U          17
Name: count, dtype: int64

job_title
unknown                  106
Associate Professor       15
Software Consultant       14
Environmental Tech        14
Chief Design Engineer     13
                        ... 
Staff Accountant II        1
Web Developer II           1
Statistician III           1
Programmer Analyst IV      1
Web Developer I            1
Name: count, Length: 185, dtype: int64

job_industry_category
Financial Services    203
Manufacturing         199
unknown               165
Health       

In [48]:
df_newcus['address'].duplicated().any()

False

In [49]:
newcus_age = df_newcus['age'].nlargest(n=5)
newcus_age

18     85
191    85
265    85
414    85
585    85
Name: age, dtype: int64

In [50]:
# change gender from U to not specify
df_newcus['gender'] = df_newcus['gender'].str.replace('U', 'not specify')
df_newcus['gender'].value_counts()

gender
Female         513
Male           470
not specify     17
Name: count, dtype: int64

In [51]:
demo_categorical = df_demo.select_dtypes(include=['object']).columns
for col in demo_categorical:
    print(df_demo[col].value_counts())
    print()

first_name
Max         5
Tobe        5
Timmie      5
Kippy       4
Pail        4
           ..
Katy        1
Hakim       1
Fanchon     1
Jeanette    1
Sarene      1
Name: count, Length: 3139, dtype: int64

last_name
unknown      125
Pristnor       3
Ramsdell       3
Gimber         2
Roelofs        2
            ... 
Whittock       1
Carwithen      1
Blas           1
Dearlove       1
Oldland        1
Name: count, Length: 3726, dtype: int64

gender
Female    2037
Male      1872
U           88
F            1
Femal        1
M            1
Name: count, dtype: int64

job_title
unknown                                 506
Business Systems Development Analyst     45
Tax Accountant                           44
Social Worker                            44
Internal Auditor                         42
                                       ... 
Database Administrator I                  4
Health Coach I                            3
Health Coach III                          3
Research Assistant III    

In [52]:
df_demo['gender'] = df_demo['gender'].str.replace('U', 'not specify')
df_demo['gender'] = df_demo['gender'].replace('F', 'Female')
df_demo['gender'] = df_demo['gender'].replace('Femal', 'Female')
df_demo['gender'] = df_demo['gender'].replace('M', 'Male')
df_demo['gender'].value_counts()

gender
Female         2039
Male           1873
not specify      88
Name: count, dtype: int64

In [53]:
demo_numerical = df_demo.select_dtypes(exclude=['object']).columns
for col in demo_numerical:
    print(df_demo[col].value_counts())
    print()

customer_id
1       1
2672    1
2659    1
2660    1
2661    1
       ..
1339    1
1340    1
1341    1
1342    1
4000    1
Name: count, Length: 4000, dtype: int64

past_3_years_bike_related_purchases
16    56
19    56
67    54
20    54
2     50
      ..
8     28
95    27
85    27
86    27
92    24
Name: count, Length: 100, dtype: int64

DOB
1977-07-12 23:56:41.277792    87
1978-01-30 00:00:00.000000     7
1962-12-17 00:00:00.000000     4
1977-05-13 00:00:00.000000     4
1976-07-16 00:00:00.000000     4
                              ..
1989-06-16 00:00:00.000000     1
1998-09-30 00:00:00.000000     1
1985-03-11 00:00:00.000000     1
1989-10-23 00:00:00.000000     1
1991-11-05 00:00:00.000000     1
Name: count, Length: 3449, dtype: int64

tenure
7.000000     235
5.000000     228
11.000000    221
10.000000    218
16.000000    215
8.000000     211
18.000000    208
12.000000    202
9.000000     200
14.000000    200
6.000000     192
4.000000     191
13.000000    191
17.000000    182
15.000000

In [54]:
df_demo['customer_id'].duplicated().any()

False

In [55]:
age = df_demo['age'].nlargest(n=5)
print(age)
demo_age_outliers = df_demo[df_demo['age'] > 100]
demo_age_outliers = len(demo_age_outliers)
print(demo_age_outliers)

33      179
719      92
1091     88
3409     83
2412     80
Name: age, dtype: int64
1


In [56]:
# outliers = df_demo[df_demo['age'] > 100]
# target_age_dob = df_demo.loc[df_demo['age'] == 92, 'DOB'].values[0]
# df_demo.loc[df_demo['age'] > 100, 'DOB'] = target_age_dob

In [57]:
# Replace with average age
# Replace 179 with NaN
df_demo['age'] = df_demo['age'].replace(179, np.nan)

# Calculate mean age excluding NaN values
mean_age = df_demo['age'].mean()

# Fill NaN values with the mean age
df_demo['age'].fillna(mean_age, inplace=True)

# Display the top 5 ages after the replacement
print(df_demo['age'].value_counts())

age
46.000000    277
45.000000    232
49.000000    149
47.000000    144
43.000000    130
44.000000    124
37.000000    119
48.000000    116
50.000000    113
28.000000     95
64.000000     92
52.000000     82
34.000000     81
38.000000     81
42.000000     81
56.000000     78
59.000000     77
26.000000     76
36.000000     75
25.000000     75
61.000000     74
58.000000     72
29.000000     71
55.000000     70
35.000000     68
31.000000     67
67.000000     67
54.000000     67
57.000000     67
30.000000     66
24.000000     66
53.000000     66
62.000000     65
27.000000     64
60.000000     62
66.000000     61
69.000000     61
63.000000     60
51.000000     59
68.000000     56
65.000000     56
33.000000     56
23.000000     47
32.000000     46
41.000000     45
40.000000     41
39.000000     40
22.000000     32
70.000000     14
21.000000     11
45.812203      1
79.000000      1
92.000000      1
88.000000      1
80.000000      1
83.000000      1
Name: count, dtype: int64


In [58]:
address_categorical = df_address.select_dtypes(include=['object']).columns
for col in address_categorical:
    print(df_address[col].value_counts())
    print()

address
3 Mariners Cove Terrace      2
3 Talisman Place             2
64 Macpherson Junction       2
359 Briar Crest Road         1
4543 Service Terrace         1
                            ..
5063 Shopko Pass             1
09 Hagan Pass                1
87897 Lighthouse Bay Pass    1
294 Lawn Junction            1
320 Acker Drive              1
Name: count, Length: 3996, dtype: int64

state
NSW                2054
VIC                 939
QLD                 838
New South Wales      86
Victoria             82
Name: count, dtype: int64

country
Australia    3999
Name: count, dtype: int64



In [59]:
df_address['address'].duplicated().any()

True

In [60]:
duplicated_rows = df_address[df_address['address'].duplicated(keep=False)]
duplicated_rows

,customer_id,address,postcode,state,country,property_valuation
732,737,3 Talisman Place,4811,QLD,Australia,2
2315,2320,64 Macpherson Junction,2208,NSW,Australia,11
2328,2333,3 Mariners Cove Terrace,3108,VIC,Australia,10
2470,2475,3 Talisman Place,4017,QLD,Australia,5
2980,2985,3 Mariners Cove Terrace,2216,NSW,Australia,10
3535,3540,64 Macpherson Junction,4061,QLD,Australia,8


In [61]:
merge_df = pd.merge(duplicated_rows,df_demo, on='customer_id', how='inner')
result_df = merge_df[['customer_id', 'first_name', 'last_name','DOB','address', 'postcode', 'state']]
result_df

,customer_id,first_name,last_name,DOB,address,postcode,state
0,737,Sisely,Oppy,1983-07-11,3 Talisman Place,4811,QLD
1,2320,Kaylyn,Throssell,1965-12-21,64 Macpherson Junction,2208,NSW
2,2333,Ives,Adolfson,1966-04-19,3 Mariners Cove Terrace,3108,VIC
3,2475,Cammi,Ambrogioni,1977-05-02,3 Talisman Place,4017,QLD
4,2985,Kerby,Nesfield,1986-10-14,3 Mariners Cove Terrace,2216,NSW
5,3540,Ali,Naris,1954-08-27,64 Macpherson Junction,4061,QLD


In [62]:
df_address['state'] = df_address['state'].str.replace('New South Wales', 'NSW')
df_address['state'] = df_address['state'].str.replace('Victoria', 'VIC')
df_address['state'].value_counts()

state
NSW    2140
VIC    1021
QLD     838
Name: count, dtype: int64

### Unnamed column

In [63]:
unname_col = df_newcus[['Unnamed: 16','Unnamed: 17','Unnamed: 18','Unnamed: 19','Unnamed: 20']]
unname_col

,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,0.56,0.7000,0.875000,0.743750,1
1,0.89,0.8900,1.112500,0.945625,1
2,1.01,1.0100,1.010000,1.010000,1
3,0.87,1.0875,1.087500,1.087500,4
4,0.52,0.5200,0.650000,0.650000,4
...,...,...,...,...,...
995,0.79,0.7900,0.790000,0.790000,996
996,0.76,0.7600,0.950000,0.807500,997
997,0.85,1.0625,1.062500,1.062500,997
998,1.09,1.3625,1.362500,1.158125,997


In [64]:
identical_col = df_newcus['Unnamed: 20'].equals(df_newcus['Rank'])
print(f"Columns are identical: {identical_col}")

Columns are identical: True


In [65]:
df_newcus = df_newcus.drop(['Unnamed: 20'], axis=1)
df_newcus

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,...,state,country,property_valuation,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Rank,Value,age
0,Chickie,Brister,Male,86,1957-07-12,General Manager,Manufacturing,Mass Customer,N,Yes,...,QLD,Australia,6,0.56,0.7000,0.875000,0.743750,1,1.718750,66
1,Morly,Genery,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,N,No,...,NSW,Australia,11,0.89,0.8900,1.112500,0.945625,1,1.718750,53
2,Ardelis,Forrester,Female,10,1974-08-28,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,...,VIC,Australia,5,1.01,1.0100,1.010000,1.010000,1,1.718750,49
3,Lucine,Stutt,Female,64,1979-01-28,Account Representative III,Manufacturing,Affluent Customer,N,Yes,...,QLD,Australia,1,0.87,1.0875,1.087500,1.087500,4,1.703125,44
4,Melinda,Hadlee,Female,34,1965-09-21,Financial Analyst,Financial Services,Affluent Customer,N,No,...,NSW,Australia,9,0.52,0.5200,0.650000,0.650000,4,1.703125,58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Ferdinand,Romanetti,Male,60,1959-10-07,Paralegal,Financial Services,Affluent Customer,N,No,...,NSW,Australia,7,0.79,0.7900,0.790000,0.790000,996,0.374000,64
996,Burk,Wortley,Male,22,2001-10-17,Senior Sales Associate,Health,Mass Customer,N,No,...,NSW,Australia,10,0.76,0.7600,0.950000,0.807500,997,0.357000,22
997,Melloney,Temby,Female,17,1954-10-05,Budget/Accounting Analyst IV,Financial Services,Affluent Customer,N,Yes,...,QLD,Australia,2,0.85,1.0625,1.062500,1.062500,997,0.357000,69
998,Dickie,Cubbini,Male,30,1952-12-17,Financial Advisor,Financial Services,Mass Customer,N,Yes,...,QLD,Australia,2,1.09,1.3625,1.362500,1.158125,997,0.357000,70


In [66]:
# df_newcus = df_newcus.rename(columns={"Unnamed: 16": "randint40-110", 
#                           "Unnamed: 17": "car ownership weight", 
#                           "Unnamed: 18": "property_val weight",
#                           "Unnamed: 19": "customer weight",
#                          }, errors="raise")
# df_newcus

## Calculate accuracy

In [67]:
# transactions null value and inconsistent id
df_trans_acc = ((len(df_transactions) - (len(inconsistent_trans_cusID) + len(rows_with_missing_values_trans))) / len(df_transactions)) * 100
print(f"Percentage of rows with missing values in df_transaction: {round(df_trans_acc, 2)}")
                
# newCustomerList only null value
# customerDemographic has over age 1 record
# customerAddress is missing some customer id
df_demo_acc = ((len(df_demo) - (count_missing_rows_demo + demo_age_outliers)) / len(df_demo)) * 100
print(f"Percentage of rows with missing values in df_demo: {round(df_demo_acc, 2)}")

df_add_acc = ((len(df_address) - (cus_id_address_diff + missing_cus_id_address)) / len(df_address)) * 100
print(f"Percentage of rows with missing values in df_address: {round(df_add_acc, 2)}")

Percentage of rows with missing values in df_transaction: 97.18
Percentage of rows with missing values in df_demo: 65.72
Percentage of rows with missing values in df_address: 99.82


## Cybersecurity risk

In [68]:
change_folder = df_demo[df_demo['default'].str.contains('../../../../../../../../../', case=False, na=False)]
xxs = df_demo[df_demo['default'].str.contains('<script>', case=False, na=False)]
xxs_ = df_demo[df_demo['default'].str.contains('/>', case=False, na=False)]
bash_inject = df_demo[df_demo['default'].str.contains('touch', case=False, na=False)]
sql_inject = df_demo[df_demo['default'].str.contains('drop table', case=False, na=False)]

In [69]:
change_folder['default']

23          ../../../../../../../../../../../etc/hosts
43      ../../../../../../../../../../../etc/passwd%00
47          ../../../../../../../../../../../etc/hosts
96          ../../../../../../../../../../../etc/hosts
126     ../../../../../../../../../../../etc/passwd%00
                             ...                      
3729    ../../../../../../../../../../../etc/passwd%00
3740    ../../../../../../../../../../../etc/passwd%00
3870        ../../../../../../../../../../../etc/hosts
3906        ../../../../../../../../../../../etc/hosts
3962    ../../../../../../../../../../../etc/passwd%00
Name: default, Length: 84, dtype: object

In [70]:
xxs['default']

1                 <script>alert('hi')</script>
32                <script>alert('hi')</script>
46                <script>alert('hi')</script>
66      <svg><script>0<1>alert('XSS')</script>
174     <svg><script>0<1>alert('XSS')</script>
                         ...                  
3760              <script>alert('hi')</script>
3772              <script>alert('hi')</script>
3815    <svg><script>0<1>alert('XSS')</script>
3828    <svg><script>0<1>alert('XSS')</script>
3864    <svg><script>0<1>alert('XSS')</script>
Name: default, Length: 70, dtype: object

In [71]:
xxs_['default']

100     <img src=x onerror=alert('hi') />
240     <img src=x onerror=alert('hi') />
306     <img src=x onerror=alert('hi') />
441     <img src=x onerror=alert('hi') />
447     <img src=x onerror=alert('hi') />
522     <img src=x onerror=alert('hi') />
707     <img src=x onerror=alert('hi') />
814     <img src=x onerror=alert('hi') />
1075    <img src=x onerror=alert('hi') />
1559    <img src=x onerror=alert('hi') />
1618    <img src=x onerror=alert('hi') />
1660    <img src=x onerror=alert('hi') />
1667    <img src=x onerror=alert('hi') />
2080    <img src=x onerror=alert('hi') />
2157    <img src=x onerror=alert('hi') />
2494    <img src=x onerror=alert('hi') />
2545    <img src=x onerror=alert('hi') />
2635    <img src=x onerror=alert('hi') />
2730    <img src=x onerror=alert('hi') />
2785    <img src=x onerror=alert('hi') />
2850    <img src=x onerror=alert('hi') />
3082    <img src=x onerror=alert('hi') />
3100    <img src=x onerror=alert('hi') />
3165    <img src=x onerror=alert('

In [72]:
bash_inject['default']

3       () { _; } >_[$($())] { touch /tmp/blns.shellsh...
28           () { 0; }; touch /tmp/blns.shellshock1.fail;
40      () { _; } >_[$($())] { touch /tmp/blns.shellsh...
88           () { 0; }; touch /tmp/blns.shellshock1.fail;
134                /dev/null; touch /tmp/blns.fail ; echo
                              ...                        
3817    () { _; } >_[$($())] { touch /tmp/blns.shellsh...
3850         () { 0; }; touch /tmp/blns.shellshock1.fail;
3900         () { 0; }; touch /tmp/blns.shellshock1.fail;
3923         () { 0; }; touch /tmp/blns.shellshock1.fail;
3968         () { 0; }; touch /tmp/blns.shellshock1.fail;
Name: default, Length: 105, dtype: object

In [73]:
sql_inject

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure,age
57,58,Dalenna,Pinnock,Female,26,1967-07-04,Office Assistant III,Financial Services,Affluent Customer,N,1;DROP TABLE users,Yes,17.0,56.0
76,77,Shermie,Andrin,Male,79,1989-09-12,Automation Specialist IV,Financial Services,Mass Customer,N,1;DROP TABLE users,No,7.0,34.0
103,104,Odille,Panketh,Female,40,1978-06-06,Automation Specialist II,Health,Mass Customer,N,1;DROP TABLE users,Yes,9.0,45.0
136,137,Vaughn,Artin,Male,27,1987-11-21,Office Assistant II,Property,High Net Worth,N,1'; DROP TABLE users--,No,10.0,35.0
243,244,Germayne,Sperry,Male,57,1974-11-25,unknown,Retail,Affluent Customer,N,1'; DROP TABLE users--,No,8.0,48.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3423,3424,Vale,Whittuck,Female,39,1977-02-28,Automation Specialist III,Manufacturing,Affluent Customer,N,1'; DROP TABLE users--,Yes,11.0,46.0
3567,3568,Berthe,Griniov,Female,13,1955-09-15,Nurse,Retail,Affluent Customer,N,1'; DROP TABLE users--,No,5.0,68.0
3736,3737,Elyssa,De Witt,Female,86,1981-03-21,Design Engineer,Manufacturing,Affluent Customer,N,1'; DROP TABLE users--,No,8.0,42.0
3918,3919,Traci,Beeckx,Female,38,1963-07-30,Cost Accountant,Financial Services,High Net Worth,N,1'; DROP TABLE users--,No,18.0,60.0


In [74]:
# df_transactions.to_csv('transaction_cleaned.csv', index = None, header=True)
df_newcus.to_csv('newcustomer_cleaned.csv', index = None, header=True)
# df_demo.to_csv('demo_cleaned.csv', index = None, header=True)
# df_address.to_csv('address_cleaned.csv', index = None, header=True)

## Demo + Address

In [75]:
merge_demo_add = pd.merge(df_demo,df_address, on='customer_id', how='left')
merge_demo_add.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure,age,address,postcode,state,country,property_valuation
0,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,"""'",Yes,11.0,70.0,060 Morning Avenue,2016.0,NSW,Australia,10.0
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,<script>alert('hi')</script>,Yes,16.0,42.0,6 Meadow Vale Court,2153.0,NSW,Australia,10.0
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,N,2018-02-01 00:00:00,Yes,15.0,69.0,NaN,NaN,NaN,NaN,NaN
3,4,Talbot,unknown,Male,33,1961-10-03,unknown,IT,Mass Customer,N,() { _; } >_[$($())] { touch /tmp/blns.shellsh...,No,7.0,62.0,0 Holy Cross Court,4211.0,QLD,Australia,9.0
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,unknown,Affluent Customer,N,NIL,Yes,8.0,46.0,17979 Del Mar Point,2448.0,NSW,Australia,4.0


In [76]:
missing_add = merge_demo_add.state.isnull().sum()
missing_add

4

In [77]:
len(merge_demo_add)

4000

In [78]:
related_cus_id = ((len(merge_demo_add) - missing_add)/len(merge_demo_add)) * 100
related_cus_id

99.9

In [79]:
merge_demo_add_ids = merge_demo_add['customer_id'].unique()
found_in_trans = df_transactions[df_transactions['customer_id'].isin(merge_demo_add_ids)]
total_customers_in_trans = len(found_in_trans['customer_id'].unique())

print(f'Total number of unique customers in the Transaction dataset: {total_customers_in_trans}')


Total number of unique customers in the Transaction dataset: 3493


In [80]:
not_found_in_trans = df_transactions[df_transactions['customer_id'].isin(merge_demo_add_ids) == False]

In [81]:
not_found_in_trans

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
8707,8708,0,5034,2017-10-07,0.0,Approved,Solex,Road,medium,medium,416.98,312.735016,2015-08-02
16700,16701,0,5034,2017-01-27,0.0,Approved,Norco Bicycles,Standard,medium,medium,360.40,270.299988,1997-08-25
17468,17469,0,5034,2017-01-03,0.0,Approved,OHM Cycles,Road,medium,medium,742.54,667.400024,1992-10-02


In [82]:
idmax_cus = merge_demo_add['customer_id'].max()
idmax_cus

4000

In [83]:
merge_demo_add_ids = merge_demo_add['customer_id'].unique()
transaction_ids = df_transactions['customer_id'].unique()

missing_customer_ids = set(merge_demo_add_ids) - set(transaction_ids)
total_missing_customers = len(missing_customer_ids)

print(f'Total number of missing customers in the Transaction dataset: {total_missing_customers}')
print(f'Missing customer IDs: {missing_customer_ids}')

Total number of missing customers in the Transaction dataset: 507
Missing customer IDs: {2074, 2660, 852, 869, 3229, 3236, 1373, 3501, 3502, 3503, 3504, 3505, 3506, 3507, 3508, 3509, 3510, 3511, 3512, 3513, 3514, 3515, 3516, 3517, 3518, 3519, 3520, 3521, 3522, 3523, 3524, 3525, 3526, 3527, 3528, 3529, 3530, 3531, 3532, 3533, 3534, 3535, 3536, 3537, 3538, 3539, 3540, 3541, 3542, 3543, 3544, 3545, 3546, 3547, 3548, 3549, 3550, 3551, 3552, 3553, 3554, 3555, 3556, 3557, 3558, 3559, 3560, 3561, 3562, 3563, 3564, 3565, 3566, 3567, 3568, 3569, 3570, 3571, 3572, 3573, 3574, 3575, 3576, 3577, 3578, 3579, 3580, 3581, 3582, 3583, 3584, 3585, 3586, 3587, 3588, 3589, 3590, 3591, 3592, 3593, 3594, 3595, 3596, 3597, 3598, 3599, 3600, 3601, 3602, 3603, 3604, 3605, 3606, 3607, 3608, 3609, 3610, 3611, 3612, 3613, 3614, 3615, 3616, 3617, 3618, 3619, 3620, 3621, 3622, 3623, 3624, 3625, 3626, 3627, 3628, 3629, 3630, 3631, 3632, 3633, 3634, 3635, 3636, 3637, 3638, 3639, 3640, 3641, 3642, 3643, 3644, 3645, 3

In [84]:
related_trans = ((idmax_cus - total_missing_customers) / idmax_cus) * 100
related_trans

87.325